In [1]:
from yolo_utils import *

#### Set parameters before training or inference

In [6]:
dynamic_dir = '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/gms_images_train'
train_with_background_images = True
test_with_background_images = True
region = 'lucknow' # lucknow, west_bengal

if region == 'lucknow':
    if train_with_background_images:
        training_set_path = '/home/shataxi.dubey/shataxi_work/vlm_on_planet/gms/lucknow_small_600_sq_km/kiln_images/train_with_background_images' # west_bengal_small_639_sq_km, lucknow_small_600_sq_km
    else:
        training_set_path = '/home/shataxi.dubey/shataxi_work/vlm_on_planet/gms/lucknow_small_600_sq_km/kiln_images/train'

    validation_set_path = '/home/shataxi.dubey/shataxi_work/vlm_on_planet/gms/lucknow_small_600_sq_km/kiln_images/valid'

    if test_with_background_images:
        test_set_path = '/home/shataxi.dubey/shataxi_work/vlm_on_planet/gms/lucknow_small_600_sq_km/kiln_images/test_with_background_images'
    else:
        test_set_path = '/home/shataxi.dubey/shataxi_work/vlm_on_planet/gms/lucknow_small_600_sq_km/kiln_images/test'

if region == 'west_bengal':
    if train_with_background_images:
        training_set_path = '/home/shataxi.dubey/shataxi_work/vlm_on_planet/gms/west_bengal_small_639_sq_km/kiln_images/train_with_background_images' # west_bengal_small_639_sq_km, lucknow_small_600_sq_km
    else:
        training_set_path = '/home/shataxi.dubey/shataxi_work/vlm_on_planet/gms/west_bengal_small_639_sq_km/kiln_images/train'

    validation_set_path = '/home/shataxi.dubey/shataxi_work/vlm_on_planet/gms/west_bengal_small_639_sq_km/kiln_images/valid'

    if test_with_background_images:
        test_set_path = '/home/shataxi.dubey/shataxi_work/vlm_on_planet/gms/west_bengal_small_639_sq_km/kiln_images/test_with_background_images'
    else:
        test_set_path = '/home/shataxi.dubey/shataxi_work/vlm_on_planet/gms/west_bengal_small_639_sq_km/kiln_images/test'

yaml_path = '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/data.yml'
type = 'png' # png, tif
image_size = 320
epochs = 50
batch_size = 4
model_id = 'yolo11m-obb.yaml'
task = 'obb'
is_dota_dataset = False
CLASSES = ['brick kilns with chimney']

# GMS imagery
gt_image_dir = f'{test_set_path}/images'
gt_label_dir = f'{test_set_path}/labels'

## class information in yml file 
data_yml_save_path=mkdtemp()
data_yml = f"""train: dummy
val: dummy
nc: 1
names: ["brick kilns with chimney"]
"""
data_yml_path = f"{data_yml_save_path}/data.yml"
with open(data_yml_path, "w") as f:
    f.write(data_yml)

## Supervision dataset
sv_dataset=sv.DetectionDataset.from_yolo(gt_image_dir,gt_label_dir,data_yml_path,is_obb=True)

In [3]:
len(glob(training_set_path+'/images/*'))

2879

#### Training

In [4]:
num_bg_image = 623
for num_non_bg_image in [98]:
    create_train_directory(dynamic_dir, training_set_path, num_non_bg_image, num_bg_image, type)
    create_yaml_file(yaml_path, dynamic_dir, validation_set_path)
    checkpoint_dir = f'{region}_{task}_{num_non_bg_image}_non_bg_{num_bg_image}_bg_{image_size}_{epochs}_{batch_size}'
    train_yolo(model_id, yaml_path, epochs, image_size, batch_size, checkpoint_dir)
    class_mapping = create_class_mapping(CLASSES, is_dota_dataset)
    model_checkpoint = f'/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/{checkpoint_dir}/weights/best.pt'

    targets, predictions, images = inference(model_checkpoint, sv_dataset, is_dota_dataset)
    map_result, map_result_50, map_result_50_95 = calculate_map(predictions, targets)
    print(f'{model_checkpoint}')
    df = calculate_confusion_matrix(predictions, targets, CLASSES, map_result)
    display(df)
    visualize_predictions(images, predictions, targets, start=0, end=25, rows=5, cols=5)

New https://pypi.org/project/ultralytics/8.3.141 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.39 🚀 Python-3.11.5 torch-2.5.1+cu124 CUDA:0 (NVIDIA A100-SXM4-80GB, 81156MiB)
engine/trainer: task=obb, mode=train, model=yolo11m-obb.yaml, data=/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/data.yml, epochs=50, time=None, patience=100, batch=4, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=lucknow_obb_98_non_bg_623_bg_320_50_42, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=False, mask_ratio=4, dropout=0.0, val=False, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnost

train: Scanning /home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/gms_images_train/labels... 98 images, 623 backgrounds, 0 corrupt: 100%|██████████| 721/721 [00:00<00:00, 1555.25it/s]

train: New cache created: /home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/gms_images_train/labels.cache


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/shataxi.dubey/shataxi_work/vlm_on_planet/gms/lucknow_small_600_sq_km/kiln_images/valid/labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:00<00:00, 16.90it/s]




                   all         25         25      0.846       0.88      0.924      0.533
Speed: 0.1ms preprocess, 9.3ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs/obb/lucknow_obb_98_non_bg_623_bg_320_50_42

0: 320x320 (no detections), 9.8ms
Speed: 0.5ms preprocess, 9.8ms inference, 0.5ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 10.0ms
Speed: 0.7ms preprocess, 10.0ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.9ms
Speed: 0.6ms preprocess, 9.9ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.8ms
Speed: 0.6ms preprocess, 9.8ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.8ms
Speed: 0.5ms preprocess, 9.8ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.8ms
Speed: 0.6ms preprocess, 9.8ms inference, 0.3ms postprocess per image 

,IoU,Precision,Recall,F1 score,TP,FP,FN,Kiln instances,mAP@50
0,0.1,0.0,0.0,0.0,0.0,0.0,25.0,25.0,0.0
0,0.3,0.0,0.0,0.0,0.0,0.0,25.0,25.0,0.0
0,0.5,0.0,0.0,0.0,0.0,0.0,25.0,25.0,0.0
0,0.7,0.0,0.0,0.0,0.0,0.0,25.0,25.0,0.0


<Figure size 1200x1000 with 2 Axes>

<Figure size 1200x1200 with 25 Axes>

#### Inference

In [7]:
lucknow_model_checkpoints = [
    # '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_1_non_bg_0_bg_320_50_4/weights/best.pt',
    # '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_2_non_bg_0_bg_320_50_4/weights/best.pt',
    # '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_3_non_bg_0_bg_320_50_4/weights/best.pt',
    # '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_4_non_bg_0_bg_320_50_4/weights/best.pt',
    # '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_5_non_bg_0_bg_320_50_4/weights/best.pt',
    # '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_7_non_bg_0_bg_320_50_4/weights/best.pt',
    # '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_10_non_bg_0_bg_320_50_4/weights/best.pt',
    # '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_20_non_bg_0_bg_320_50_4/weights/best.pt',
    # '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_30_non_bg_0_bg_320_50_4/weights/best.pt',
    # '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_40_non_bg_0_bg_320_50_4/weights/best.pt',
    # '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_60_non_bg_0_bg_320_50_4/weights/best.pt',
    # '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_98_non_bg_0_bg_320_50_32/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_98_non_bg_50_bg_320_50_4/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/lucknow_obb_98_non_bg_623_bg_320_50_4/weights/best.pt',
]

wb_model_checkpoints = [
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/west_bengal_obb_1_non_bg_0_bg_320_50_4/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/west_bengal_obb_2_non_bg_0_bg_320_50_4/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/west_bengal_obb_3_non_bg_0_bg_320_50_4/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/west_bengal_obb_4_non_bg_0_bg_320_50_4/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/west_bengal_obb_5_non_bg_0_bg_320_50_4/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/west_bengal_obb_7_non_bg_0_bg_320_50_4/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/west_bengal_obb_10_non_bg_0_bg_320_50_4/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/west_bengal_obb_20_non_bg_0_bg_320_50_4/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/west_bengal_obb_30_non_bg_0_bg_320_50_4/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/west_bengal_obb_40_non_bg_0_bg_320_50_4/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/west_bengal_obb_60_non_bg_0_bg_320_50_4/weights/best.pt',
    '/home/shataxi.dubey/shataxi_work/vlm_on_planet/Yolo/runs/obb/west_bengal_obb_98_non_bg_0_bg_320_50_4/weights/best.pt',
]

for model_checkpoint in lucknow_model_checkpoints:
    targets, predictions, images = inference(model_checkpoint, sv_dataset, is_dota_dataset)
    print(f'{model_checkpoint}')
    map_result, map_result_50, map_result_50_95 = calculate_map(predictions, targets)
    df = calculate_confusion_matrix(predictions, targets, CLASSES, map_result)
    display(df)
    # visualize_predictions(images, predictions, targets, start=0, end=25, rows=5, cols=5)


0: 320x320 9.9ms
Speed: 2.8ms preprocess, 9.9ms inference, 3.6ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 10.2ms
Speed: 0.5ms preprocess, 10.2ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.8ms
Speed: 0.6ms preprocess, 9.8ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.8ms
Speed: 0.6ms preprocess, 9.8ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.8ms
Speed: 0.6ms preprocess, 9.8ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.8ms
Speed: 0.6ms preprocess, 9.8ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 10.7ms
Speed: 0.6ms preprocess, 10.7ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 12.3ms
Speed: 0.8ms preprocess, 12.3ms inference, 0.4ms postprocess

,IoU,Precision,Recall,F1 score,TP,FP,FN,Kiln instances,mAP@50
0,0.1,0.171875,0.88,0.287582,22.0,106.0,3.0,25.0,0.872558
0,0.3,0.171875,0.88,0.287582,22.0,106.0,3.0,25.0,0.872558
0,0.5,0.171875,0.88,0.287582,22.0,106.0,3.0,25.0,0.872558
0,0.7,0.093750,0.48,0.156863,12.0,116.0,13.0,25.0,0.872558



0: 320x320 (no detections), 9.7ms
Speed: 0.5ms preprocess, 9.7ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.8ms
Speed: 0.6ms preprocess, 9.8ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.9ms
Speed: 0.7ms preprocess, 9.9ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.8ms
Speed: 0.4ms preprocess, 9.8ms inference, 0.4ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.8ms
Speed: 0.6ms preprocess, 9.8ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.8ms
Speed: 0.6ms preprocess, 9.8ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.8ms
Speed: 0.6ms preprocess, 9.8ms inference, 0.3ms postprocess per image at shape (1, 3, 320, 320)

0: 320x320 (no detections), 9.7ms
Speed: 0.6ms preprocess, 9.7ms inference, 0.4ms 

,IoU,Precision,Recall,F1 score,TP,FP,FN,Kiln instances,mAP@50
0,0.1,0.370370,0.80,0.506329,20.0,34.0,5.0,25.0,0.646459
0,0.3,0.370370,0.80,0.506329,20.0,34.0,5.0,25.0,0.646459
0,0.5,0.351852,0.76,0.481013,19.0,35.0,6.0,25.0,0.646459
0,0.7,0.185185,0.40,0.253165,10.0,44.0,15.0,25.0,0.646459
